In [7]:
import pandas as pd
import time
import random
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import math

In [2]:
df = pd.read_json("job_postings_prepped.json")

In [6]:
import pickle
vectorizer = pickle.load(open('vectorizer.pkl','rb'))

## LSH **Calculations**


In [45]:
import json

k = 10 #TODO: find out optimal value
vector_dimensions = len(df['Vector'][0])
non_zero_values = len(df['Vector'][0]) #TODO: find out optimal value

def generate_hash_function(k,vector_dimensions,non_zero_values):
    hash_functions_list = []

    for i in range(k):
        if i%10 == 0: print("Progress:", str(i)+"/"+str(k))
        hash_functions = np.zeros(vector_dimensions)
        selected_indexes = random.sample(range(vector_dimensions),non_zero_values)
        
        for index in selected_indexes:
            random_vector = np.random.randn(1)
            hash_functions[index] = random_vector

        hash_functions_list.append(hash_functions)

    return hash_functions_list

random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
random_hash_functions_df = pd.DataFrame({'Hash_Functions': random_hash_functions})
random_hash_functions_df

# random_hash_functions_df.to_json('LSH Search Results/LSH 10 Random Hash Functions.json', index=False)


Progress: 0/10


C:\Users\ragne\AppData\Local\Temp\ipykernel_22204\3049473867.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hash_functions[index] = random_vector


,Hash_Functions
0,"[-0.4621593152243012, -0.35672951157632493, 1...."
1,"[0.47087120585137815, 1.401192565415804, 0.946..."
2,"[-0.9023205094063987, -0.9520906006374974, -0...."
3,"[0.27129118011215153, -0.9232229045554883, 0.5..."
4,"[-0.14656172513477275, -0.39618645740627173, 1..."
5,"[0.8966847254456627, -0.5544146627707663, -1.8..."
6,"[0.6977282567135773, -0.540626667009899, 0.286..."
7,"[-0.894841459583318, -1.5152027984681657, 0.61..."
8,"[0.798601012942948, 0.2437842899682288, -0.557..."
9,"[-0.7675097294221432, -0.5598300776180332, -1...."


In [3]:
# import time

# Perform dot product and get binary vectors
def generate_binary_vectors(input_vector,random_hash_functions_df):
    binary_vector = []
    for i,row in random_hash_functions_df.iterrows():
        input_vector = np.asarray(input_vector, dtype=np.float64)
        hash_function = np.asarray(row['Hash_Functions'], dtype=np.float64)
    
        dot_product = np.dot(input_vector,hash_function)
        # result = 0 if dot_product <= 0 else 1
        result = np.where(dot_product <= 0, 0, 1)
        binary_vector.append(result.tolist())
    return binary_vector

# Make a new column called Binary Vectors for all the binary vectors
# random_hash_functions_df = pd.read_json('LSH Search Results/LSH Random Hash Functions.json')
# df['Binary Vectors'] = df['Vector'].apply(lambda x : generate_binary_vectors(x,random_hash_functions_df))

In [56]:
import json
test_hash = pd.DataFrame({'Hash_Functions': generate_hash_function(2, 105248, 105248)})
generate_binary_vectors(json.loads(df.iloc[0]["Vector"]), test_hash)
# len(query_matrix[0])
# json.loads(df.iloc[0]["Vector"])

C:\Users\Admin\AppData\Local\Temp\ipykernel_14292\3669591667.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hash_functions[index] = random_vector


Time Taken to generate binary vectors: 0.003000020980834961


[0, 1]

In [4]:
# Perform linear search on binary vector using hamming distance Dh
def hamming_distance(query_vector,database_sample):
  mismatched_bits = 0
  for i in range(len(query_vector)):
    # print(i)
    # print('query vector = ',query_vector[i])
    # print('database sample = ',database_sample[i])
    if query_vector[i] != database_sample[i]:
      mismatched_bits += 1
  return mismatched_bits

##### Find Best K Value Time from 10 to 100

In [ ]:
#Calculate K Value Time Comparison
# k_time_dict = {}
# vector_dimensions = len(df['Vector'][0])
# non_zero_values = len(df['Vector'][0]) #TODO: find out optimal value

# for k in range(10,101,10):
#     start = time.time()

#     random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
#     binary_vectors_list = df['Vector'].apply( lambda x : one_hashing_function(x,random_hash_functions))

#     end = time.time()
#     results_time = end - start
#     k_time_dict[k] = results_time
#     k_df = pd.DataFrame(list(k_time_dict.items()), columns=['K Value', 'Time'])
#     k_df.to_csv(f'LSH Search Results/K_Value_Time.csv', index=False)


### Calculate binary of dataset

In [5]:
import time
import json

# Import Hash functions *
random_hash_functions_df = pd.read_json('LSH Search Results/LSH 300 Random Hash Functions.json')
print("Imported  hash functions from Json file")
# df['Vector'] = df['Vector'].apply(lambda x : json.loads(x))

for i in range(10,300,20):
    start_time = time.time()
    i_size_random_hash_functions = random_hash_functions_df.sample(i)
    print("Index:",list(i_size_random_hash_functions.index))
    # # Do LSH Calculations for dataset *
    df['Binary Vectors'] = df['Vector'].apply( lambda x : generate_binary_vectors(x,i_size_random_hash_functions))
    end_time = time.time()
    print("Time Taken to generate binary vectors(k =",str(i)+"):", end_time-start_time, "secs")
    print("Binary vectors successfully added, Exporting to Json...")
    df.to_csv('LSH Search Results/LSH Database with Binary Vectors (k is '+str(i)+').csv')

Imported  hash functions from Json file
Index: [159, 215, 144, 31, 0, 33, 210, 299, 227, 133]
Time Taken to generate binary vectors(k = 10): 340.71075892448425 secs
Binary vectors successfully added, Exporting to Json...
Index: [56, 59, 46, 43, 84, 227, 172, 5, 165, 52, 92, 213, 176, 17, 67, 77, 96, 131, 175, 110, 254, 41, 66, 290, 287, 58, 6, 219, 4, 32]
Time Taken to generate binary vectors(k = 30): 935.3069818019867 secs
Binary vectors successfully added, Exporting to Json...
Index: [243, 60, 115, 254, 59, 296, 19, 299, 186, 155, 272, 257, 141, 54, 149, 266, 209, 193, 225, 221, 287, 146, 293, 196, 75, 67, 276, 175, 295, 142, 104, 22, 49, 114, 42, 203, 58, 6, 292, 55, 108, 82, 286, 217, 69, 268, 91, 48, 30, 93]
Time Taken to generate binary vectors(k = 50): 1529.8507180213928 secs
Binary vectors successfully added, Exporting to Json...
Index: [8, 53, 289, 133, 226, 21, 216, 295, 268, 296, 214, 165, 218, 61, 121, 243, 62, 140, 16, 143, 183, 223, 262, 30, 55, 110, 284, 91, 89, 77, 95, 

## Calculate 10 different queries accuracies

In [6]:
# Calculate 10 different queries accuracies
import os

# Defining queries
query_dict = {'Q1': "resilient investment banker", 
              'Q2': "2 years experience product manager", 
              'Q3': "10 years risk analyst problem solver", 
              'Q4': "tax analyst for big company", 
              'Q5': "software engineer for google or amazon", 
              'Q6': "video editor for advertisements with 5 year experience",
              'Q7': "full time senior head nurse position",
              'Q8': "after school math and science tutor",
              'Q9': "dietitian for professional atheletes",
              'Q10': "costume designer and makeup artist"}

query_df = pd.DataFrame.from_dict(query_dict, orient='index', columns=['Query Text'])

# Using the imported count vectorizer to generate counter vector of queries (N)
query_vectors = vectorizer.transform(list(query_dict.values()))
query_matrix = query_vectors.toarray()

# Import Hash functions *
random_hash_functions_df = pd.read_json('LSH Search Results/LSH Random Hash Functions.json')

# Do LSH Calculations for query vectors *
query_dict_binary_vectors = [generate_binary_vectors(vector, random_hash_functions_df) for vector in query_matrix]

query_df['Vectors'] = [row.tolist() for row in query_matrix]
query_df['Binary Vectors'] = query_dict_binary_vectors

query_df.to_csv('LSH Search Results/LSH Query Vectors with Binary Vectors.csv')

Time Taken to generate binary vectors: 0.18794512748718262
Time Taken to generate binary vectors: 0.1875159740447998
Time Taken to generate binary vectors: 0.18975400924682617
Time Taken to generate binary vectors: 0.18224525451660156
Time Taken to generate binary vectors: 0.17214012145996094
Time Taken to generate binary vectors: 0.20138168334960938
Time Taken to generate binary vectors: 0.180006742477417
Time Taken to generate binary vectors: 0.1985175609588623
Time Taken to generate binary vectors: 0.19063329696655273
Time Taken to generate binary vectors: 0.19053316116333008


In [43]:
# Calculate 10 different queries accuracies
import os

# Defining queries
query_dict = {'Q1': "resilient investment banker", 
              'Q2': "2 years experience product manager", 
              'Q3': "10 years risk analyst problem solver", 
              'Q4': "tax analyst for big company", 
              'Q5': "software engineer for google or amazon", 
              'Q6': "video editor for advertisements with 5 year experience",
              'Q7': "full time senior head nurse position",
              'Q8': "after school math and science tutor",
              'Q9': "dietitian for professional atheletes",
              'Q10': "costume designer and makeup artist"}

query_df = pd.DataFrame.from_dict(query_dict, orient='index', columns=['Query Text'])

# Using the imported count vectorizer to generate counter vector of queries (N)
query_vectors = vectorizer.transform(list(query_dict.values()))
query_matrix = query_vectors.toarray()


# Import Hash functions *
index = [27, 5, 214, 222, 224, 2, 245, 285, 191, 246, 184, 35, 210, 269, 264, 227, 177, 172, 114, 244, 225, 211, 202, 295, 121, 230, 215, 139, 44, 132, 89, 9, 148, 103, 43, 13, 175, 155, 223, 67, 135, 76, 99, 49, 152, 163, 267, 178, 26, 66, 257, 119, 56, 136, 91, 190, 187, 75, 188, 259, 131, 143, 78, 123, 120, 176, 255, 294, 22, 149, 162, 29, 166, 54, 32, 156, 287, 109, 278, 85, 69, 256, 238, 73, 265, 234, 141, 241, 198, 142, 286, 52, 107, 77, 129, 251, 127, 186, 57, 212, 96, 167, 101, 146, 102, 170, 45, 272, 83, 203, 160, 242, 95, 154, 159, 137, 209, 104, 100, 30, 0, 68, 217, 1, 266, 15, 282, 165, 23, 125, 206, 50, 71, 122, 168, 270, 144, 213, 25, 196, 158, 62, 235, 118, 20, 237, 128, 293, 60, 233, 40, 218, 173, 105, 37, 124, 151, 220, 231, 110, 181, 8, 117, 31, 24, 94, 72, 47, 111, 157, 185, 90, 33, 252, 61, 84, 219, 290, 88, 277, 221, 254, 130, 11, 80, 74, 268, 289, 36, 194, 4, 97, 34, 279, 115, 283, 28, 82, 204, 299, 64, 195, 19, 297, 161, 182, 70, 273, 106, 281, 7, 63, 134, 79, 39, 150, 10, 53, 298, 171, 262, 46, 108, 174, 58, 296, 280, 147, 98, 236, 200, 260, 288, 145, 248, 140, 133, 3, 48, 164, 179, 291, 65, 253, 113, 169, 271, 232, 243, 263, 284, 138, 189, 41, 51, 14, 275, 6, 199, 21, 192, 93, 207, 87, 258, 180, 81, 38, 247, 16, 292, 59, 112, 42, 250, 201, 249, 183, 17, 116, 239, 92, 274, 226, 229, 193, 197, 228, 216, 18]
k = len(index)
random_hash_functions_df = pd.read_json('LSH Search Results/LSH 300 Random Hash Functions.json').iloc[index]

# Do LSH Calculations for query vectors *
query_dict_binary_vectors = [generate_binary_vectors(vector, random_hash_functions_df) for vector in query_matrix]

query_df['Vectors'] = [row.tolist() for row in query_matrix]
query_df['Binary Vectors'] = query_dict_binary_vectors

filename = f'LSH Search Results\LSH Query Vector With Binary Vectors\LSH Query Vectors with Binary Vectors(k={k}).csv'
query_df.to_csv(filename)
print(len(query_df['Binary Vectors'][0]))

290


C:\Users\ragne\AppData\Local\Temp\ipykernel_22204\1163304622.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(len(query_df['Binary Vectors'][0]))


In [29]:
query_df

,Query Text,Vectors,Binary Vectors
Q1,resilient investment banker,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 1, 1, 1, 1, 0]"
Q2,2 years experience product manager,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 0, 1, 1, 1, 0, 0]"
Q3,10 years risk analyst problem solver,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 0, 1, 0, 0]"
Q4,tax analyst for big company,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1]"
Q5,software engineer for google or amazon,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 1, 1, 0, 0]"
Q6,video editor for advertisements with 5 year ex...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 0]"
Q7,full time senior head nurse position,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 1, 1, 0, 0, 0, 0]"
Q8,after school math and science tutor,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 1, 1, 0, 1]"
Q9,dietitian for professional atheletes,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 1, 1, 0, 1]"
Q10,costume designer and makeup artist,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0]"


In [38]:
random_hash_functions_df

,Hash_Functions
0,"[0.0652451414, -0.3105812535, 0.1707870626, -0..."
1,"[-1.3519581269, -0.2255419904, 0.1975276205, -..."
2,"[0.7905945027, -0.7924027689, 0.9140652319, -0..."
3,"[-0.2492531758, 2.1408192351, -1.5438940171, -..."
4,"[1.5068522531, -0.43001041840000004, -1.153862..."
...,...
295,"[0.3290855109, -0.18241482, 0.9899872848000001..."
296,"[1.0772217896, -0.2121105697, 1.1417691648, -0..."
297,"[2.1115217074, 0.5449863267, 0.7089783419, -0...."
298,"[-0.15233194590000002, 1.6498596557, 2.5377461..."


In [64]:
import json

k=30
query_df = pd.read_csv(f'LSH Search Results/LSH Query Vectors with Binary Vectors(k={k}).csv')
df = pd.read_csv(f'LSH Search Results/LSH Database with Binary Vectors (k is {k}).csv')

query_df['Binary Vectors'] = query_df['Binary Vectors'].apply(lambda x : json.loads(x))
df['Binary Vectors'] = df['Binary Vectors'].apply(lambda x : json.loads(x))

In [ ]:
print(len(query_df['Binary Vectors'][0]))
print(len(df['Binary Vectors'][0]))

In [76]:
query_df['Query Text'][0]

'resilient investment banker'

In [44]:
import time
import json
import pandas as pd

for k in range(10,300,20):
    query_df = pd.read_csv(f'LSH Search Results\LSH Query Vector With Binary Vectors\LSH Query Vectors with Binary Vectors(k={k}).csv')
    df = pd.read_csv(f'D:\Data Analysis\LSH Database with Binary Vectors\LSH Database with Binary Vectors (k is {k}).csv')

    query_df['Binary Vectors'] = query_df['Binary Vectors'].apply(lambda x : json.loads(x))
    df['Binary Vectors'] = df['Binary Vectors'].apply(lambda x : json.loads(x))
        
    for query_index,query_row in query_df.iterrows():

        # Specify the folder path
        # folder_path = f'LSH Search Results\LSH Search Results with K Values'

        mismatched_list = []

        start_time = time.time()
        for i, row in df.iterrows():
            mismatched_list.append(hamming_distance(query_row['Binary Vectors'],row['Binary Vectors'])) 
        end_time = time.time()
        print("Duration taken to calculate hamming distance of '"+ str(query_index) +"' with dataset:", end_time-start_time)
        
        df['Distance'] = mismatched_list

        # Get only data and distance from query_name
        query_result_df = df[['Data', 'Distance']].copy()

        filename = f'LSH Search Results\LSH Search Results with K Values\LSH Search Results (k = {k})(Query = {query_df["Query Text"][query_index]}).csv'
        query_result_df.to_csv(filename)


Duration taken to calculate hamming distance of '0' with dataset: 1.5441560745239258
Duration taken to calculate hamming distance of '1' with dataset: 0.6484711170196533
Duration taken to calculate hamming distance of '2' with dataset: 0.6557807922363281
Duration taken to calculate hamming distance of '3' with dataset: 0.6359128952026367
Duration taken to calculate hamming distance of '4' with dataset: 0.646578311920166
Duration taken to calculate hamming distance of '5' with dataset: 0.6443042755126953
Duration taken to calculate hamming distance of '6' with dataset: 0.6614692211151123
Duration taken to calculate hamming distance of '7' with dataset: 0.6201503276824951
Duration taken to calculate hamming distance of '8' with dataset: 0.648831844329834
Duration taken to calculate hamming distance of '9' with dataset: 0.6394858360290527
Duration taken to calculate hamming distance of '0' with dataset: 1.2146499156951904
Duration taken to calculate hamming distance of '1' with dataset: 0